In [233]:
import numpy as np
import heapq
import copy
from typing import Callable

In [234]:
def parse_input(input_file: str) -> np.ndarray:
    with open(input_file) as risk_grid:
        return np.array([list(map(int, [*line.strip()])) 
                            for line in risk_grid.readlines()])

In [235]:
def sum_tuples(tuple_1: tuple[int], tuple_2: tuple[int]) -> tuple[int]:
    return (tuple_1[0] + tuple_2[0], tuple_1[1] + tuple_2[1])

In [236]:
def find_adjacent_coordinates(coord: tuple[int, int], max_row_index: int, max_column_index: int) -> list[tuple[int]]:
    possible_directions = [(0, i) for i in (-1, 1)] + [(j, 0) for j in (-1, 1)]
    directions_and_coords = []
    for direction in possible_directions:
        new_row, new_column = sum_tuples(coord, direction)
        if  0 <= new_row <= max_row_index and 0 <= new_column <= max_column_index:
            directions_and_coords.append(((new_row, new_column), direction))
    return directions_and_coords

In [237]:
def find_risk_level(path: tuple[list, tuple]) -> int:
    return path[1]

In [238]:
def find_euclidean_distance(point_a: tuple[int], point_b: tuple[int]):
    return ((point_a[0] - point_b[0])**2 + (point_a[1] - point_b[1])**2)**0.5

In [239]:
#First attempt, takes far too long
def find_shortest_risk_level_v1(input_file: str) -> int:
    risk_grid = parse_input(input_file)
    start_point = (0,0)
    end_point = tuple(coord - 1 for coord in risk_grid.shape)
    max_row, max_column = end_point
    current_paths = [([start_point], 0)]
    while current_paths != []:
        current_path, current_risk_level = current_paths.pop()
        current_coord = current_path[-1]
        current_distance = find_euclidean_distance(current_coord, end_point)
        if current_coord == end_point:
            return current_risk_level
        for coord in find_adjacent_coordinates(current_coord, max_row, max_column):
            new_distance = find_euclidean_distance(coord, end_point)
            if new_distance < current_distance and coord not in current_path:  
                new_path = current_path + [coord]
                updated_risk_level = current_risk_level + risk_grid[coord]
                current_paths.append((new_path, updated_risk_level))
        current_paths.sort(key=find_risk_level, reverse=True) 

In [247]:
#try using heapq and not saving whole list of coords, just where it is, how many steps, direction it got there 
def find_shortest_risk_level_v2(input_file: str, parse_input_func: Callable) -> int:
    
    risk_grid = parse_input_func(input_file)
    start_point = (0,0)
    initial_risk_level = 0
    initial_direction = (0,0)
    inital_no_steps = 0
    end_point = tuple(coord - 1 for coord in risk_grid.shape)
    max_row, max_column = end_point
    current_paths = [(initial_risk_level, start_point, initial_direction, inital_no_steps)]
    heapq.heapify(current_paths)
    seen_paths = {(initial_risk_level, start_point, initial_direction, inital_no_steps)}
    
    while current_paths:
        current_risk_level, current_coord, _, current_no_steps = heapq.heappop(current_paths)
        if current_coord == end_point:
            return current_risk_level
        for coord_and_direction in find_adjacent_coordinates(current_coord, max_row, max_column):
            new_coord, new_direction = coord_and_direction
            if coord_and_direction not in seen_paths:
                seen_paths.add(coord_and_direction)  
                new_no_steps = current_no_steps + 1
                updated_risk_level = current_risk_level + risk_grid[new_coord]
                new_path = (updated_risk_level, new_coord, new_direction, new_no_steps)
                heapq.heappush(current_paths, new_path)

In [248]:
find_shortest_risk_level_v2('practise_input.txt', parse_input)

40

In [249]:
find_shortest_risk_level_v2('real_input.txt', parse_input)

626

Part 2

In [250]:
risk_grid = parse_input('practise_input.txt')

In [251]:
def parse_input_part_2(input_file: str) -> np.ndarray:
    original_grid = parse_input(input_file)
    new_grid = copy.deepcopy(original_grid)
    final_grid = copy.deepcopy(original_grid)
    for i in range(4):
        new_grid += 1
        new_grid[new_grid==10] = 1
        final_grid = np.append(final_grid, new_grid, axis=0)
    new_grid = copy.deepcopy(final_grid)
    for j in range(4):
        new_grid += 1
        new_grid[new_grid==10] = 1
        final_grid = np.append(final_grid, new_grid, axis=1)
    return final_grid
        

In [252]:
find_shortest_risk_level_v2('practise_input.txt', parse_input_part_2)

315

In [253]:
find_shortest_risk_level_v2('real_input.txt', parse_input_part_2)

2966